In [1]:
import pandas as pd
import numpy as np
!pip install xgboost
!pip install lightgbm
!pip install imblearn

In [2]:
df1=pd.read_csv('Survival_time_event.csv', index_col=0)
df2=pd.read_csv('Treatment.csv', index_col=0)
df3=pd.read_csv('Clinical_Variables.csv', index_col=0)
df4=pd.read_csv('Genetic_alterations.csv', index_col=0)

In [3]:
data_path = './'

clinic_table = df3.to_numpy()[:,:]
genetic_table = df4.to_numpy()[:,:]
survival_table = df1.to_numpy()[:,:]
treatment_table = df2.to_numpy()[:,:]

treatment = 1
no_treatmet = 0
treatment_index = np.where(treatment==treatment_table)[0] # 치료를 받은 사람들의 Index (총 488명)
no_treatment_index = np.where(no_treatmet==treatment_table)[0] # 치료를 받지 않은 사람들의 Index (총 512명)
print('Treatment :', treatment_index.shape[0])
print('No Treatment :', no_treatment_index.shape[0])
print('Total :', treatment_index.shape[0] + no_treatment_index.shape[0])
print()

survive = 0
no_survive = 1
survive_index = np.where(survive==survival_table[:,1])[0] # 생존한 사람들의 Index (총 109명)
no_survive_index = np.where(no_survive==survival_table[:,1])[0] # 생존하지 못 한 사람들의 Index (총 891명)
print('Survive :', survive_index.shape[0])
print('No Survive :', no_survive_index.shape[0])
print('Total :', survive_index.shape[0] + no_survive_index.shape[0])
print()

treatment_survive_index = np.intersect1d(treatment_index, survive_index) # 치료 O, 생존 O (총 48명)
treatment_no_survive_index = np.intersect1d(treatment_index, no_survive_index) # 치료 O, 생존 X (총 440명)
no_treatment_survive_index = np.intersect1d(no_treatment_index, survive_index) # 치료 X, 생존 O (총 61명)
no_treatment_no_survive_index = np.intersect1d(no_treatment_index, no_survive_index) # 치료 X, 생존 X (총 451명)
print('Treatment & Survive :', treatment_survive_index.shape[0])
print('Treatment & No Survive :', treatment_no_survive_index.shape[0])
print('No Treatment & Survive :', no_treatment_survive_index.shape[0])
print('No Treatment & No Survive :', no_treatment_no_survive_index.shape[0])
print('Total :', treatment_survive_index.shape[0] + treatment_no_survive_index.shape[0] + no_treatment_survive_index.shape[0] + no_treatment_no_survive_index.shape[0])
print()

Treatment : 488
No Treatment : 512
Total : 1000

Survive : 109
No Survive : 891
Total : 1000

Treatment & Survive : 48
Treatment & No Survive : 440
No Treatment & Survive : 61
No Treatment & No Survive : 451
Total : 1000



In [4]:
# TandS = ['newlabel']
# df5 = pd.DataFrame(columns=TandS, index=range(0,1000))
# print(df5)
# df5.iloc[treatment_survive_index,0] = [0] * treatment_survive_index.shape[0]
# df5.iloc[treatment_no_survive_index,0] = [1] * treatment_no_survive_index.shape[0]
# df5.iloc[no_treatment_survive_index,0] = [2] * no_treatment_survive_index.shape[0]
# df5.iloc[no_treatment_no_survive_index,0] = [3] * no_treatment_no_survive_index.shape[0]

# print(df5)
# df5.to_csv('newLabel.csv')

In [5]:
df5=pd.read_csv('newLabel.csv', index_col=0)

In [6]:
df = pd.concat([df5,df1,df2,df3,df4], axis=1)

In [7]:
'''
Analyze by describing data
'''
'''
label class

치료o + 사망x -> class 1
나머지 경우 -> class 0

'''

df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

['newlabel' 'time' 'Var1' 'Var2' 'Var3' 'Var4' 'Var5' 'Var6' 'Var7' 'Var8'
 'Var9' 'Var10' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6' 'G7' 'G8' 'G9' 'G10' 'G11'
 'G12' 'G13' 'G14' 'G15' 'G16' 'G17' 'G18' 'G19' 'G20' 'G21' 'G22' 'G23'
 'G24' 'G25' 'G26' 'G27' 'G28' 'G29' 'G30' 'G31' 'G32' 'G33' 'G34' 'G35'
 'G36' 'G37' 'G38' 'G39' 'G40' 'G41' 'G42' 'G43' 'G44' 'G45' 'G46' 'G47'
 'G48' 'G49' 'G50' 'G51' 'G52' 'G53' 'G54' 'G55' 'G56' 'G57' 'G58' 'G59'
 'G60' 'G61' 'G62' 'G63' 'G64' 'G65' 'G66' 'G67' 'G68' 'G69' 'G70' 'G71'
 'G72' 'G73' 'G74' 'G75' 'G76' 'G77' 'G78' 'G79' 'G80' 'G81' 'G82' 'G83'
 'G84' 'G85' 'G86' 'G87' 'G88' 'G89' 'G90' 'G91' 'G92' 'G93' 'G94' 'G95'
 'G96' 'G97' 'G98' 'G99' 'G100' 'G101' 'G102' 'G103' 'G104' 'G105' 'G106'
 'G107' 'G108' 'G109' 'G110' 'G111' 'G112' 'G113' 'G114' 'G115' 'G116'
 'G117' 'G118' 'G119' 'G120' 'G121' 'G122' 'G123' 'G124' 'G125' 'G126'
 'G127' 'G128' 'G129' 'G130' 'G131' 'G132' 'G133' 'G134' 'G135' 'G136'
 'G137' 'G138' 'G139' 'G140' 'G141' 'G142' 'G143' 'G14

In [8]:
print(df.head())

   newlabel       time  Var1  Var2  Var3  Var4  Var5  Var6  Var7  Var8  ...  \
0         3  57.448331     5     1     1     4     6     5     2     1  ...   
1         3  27.004439     3     1     3     9     1     1     2     4  ...   
2         1  43.770511     2     5     3     4     3     3     3     2  ...   
3         1  32.281018     2     7     2     3     5     0     1     4  ...   
4         2  44.559284     1     3     0     0     2     2     6     3  ...   

   G291  G292  G293  G294  G295  G296  G297  G298  G299  G300  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0     0  
3     0     1     0     1     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     1  

[5 rows x 312 columns]


In [9]:
print(df.tail())

     newlabel       time  Var1  Var2  Var3  Var4  Var5  Var6  Var7  Var8  ...  \
995         2  19.289036     3     5     3     7     0     2     1     4  ...   
996         1  66.591235     4     2     1     2     2     2     2     1  ...   
997         0  62.986021     4     3     4     9     3     6     6     4  ...   
998         3  32.736220     4     1     4     5     6     3     1     2  ...   
999         3  36.714493     3     2     1     2     4     3     1     2  ...   

     G291  G292  G293  G294  G295  G296  G297  G298  G299  G300  
995     0     0     0     0     0     0     1     0     0     0  
996     0     0     1     0     1     0     0     0     1     0  
997     0     0     0     0     1     0     0     0     0     0  
998     0     0     0     0     0     0     0     1     1     0  
999     0     0     0     0     0     0     0     0     0     0  

[5 rows x 312 columns]


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
'''
Train & Test Split
'''
random_index = np.arange(1000)
# np.random.shuffle(random_index)
# np.random.shuffle(random_index)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

In [12]:
print(train_df.shape)
print(train_df.head())

(800, 312)
   newlabel       time  Var1  Var2  Var3  Var4  Var5  Var6  Var7  Var8  ...  \
0         3  57.448331     5     1     1     4     6     5     2     1  ...   
1         3  27.004439     3     1     3     9     1     1     2     4  ...   
2         1  43.770511     2     5     3     4     3     3     3     2  ...   
3         1  32.281018     2     7     2     3     5     0     1     4  ...   
4         2  44.559284     1     3     0     0     2     2     6     3  ...   

   G291  G292  G293  G294  G295  G296  G297  G298  G299  G300  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0     0  
3     0     1     0     1     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     1  

[5 rows x 312 columns]


In [13]:
print(train_df.describe())

         newlabel        time        Var1        Var2        Var3        Var4  \
count  800.000000  800.000000  800.000000  800.000000  800.000000  800.000000   
mean     1.906250   51.286198    2.918750    3.326250    3.022500    3.155000   
std      1.032615   20.840875    1.878579    1.947962    1.907627    1.943115   
min      0.000000    7.070708    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   37.385295    2.000000    2.000000    2.000000    2.000000   
50%      2.000000   47.141058    3.000000    3.000000    3.000000    3.000000   
75%      3.000000   60.610372    4.000000    4.000000    4.000000    4.000000   
max      3.000000  171.994623    9.000000   11.000000   11.000000   12.000000   

             Var5        Var6        Var7        Var8  ...       G291  \
count  800.000000  800.000000  800.000000  800.000000  ...  800.00000   
mean     3.470000    3.265000    2.346250    2.387500  ...    0.10875   
std      1.873763    2.028526    1.795938    1.8389

In [14]:
print(test_df.describe())

         newlabel        time        Var1       Var2        Var3        Var4  \
count  200.000000  200.000000  200.000000  200.00000  200.000000  200.000000   
mean     1.950000   54.156377    3.125000    3.19500    2.915000    3.440000   
std      1.059771   26.732912    1.848366    1.97903    1.943363    2.026557   
min      0.000000   -7.945621    0.000000    0.00000    0.000000    0.000000   
25%      1.000000   37.611163    2.000000    2.00000    2.000000    2.000000   
50%      2.000000   46.796557    3.000000    3.00000    2.000000    3.000000   
75%      3.000000   62.989101    4.000000    4.00000    4.000000    5.000000   
max      3.000000  217.078908   10.000000   10.00000    9.000000    9.000000   

            Var5        Var6        Var7        Var8  ...        G291  \
count  200.00000  200.000000  200.000000  200.000000  ...  200.000000   
mean     3.56500    3.120000    2.415000    2.420000  ...    0.125000   
std      2.06813    2.072941    1.913395    1.791549  ...   

In [15]:
'''
Data & Ground Truth Split
'''

'''
Time data drop
'''

X_train = train_df.drop(['time', 'newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['time', 'newlabel'], axis=1)
Y_test = test_df['newlabel']

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(800, 310) (800,) (200, 310) (200,)


In [16]:
'''
Save train & test results

When doing train, Shuffle and split the data

'''

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score


test_method_list = ['LogisticRegression', 'Support Vector Machines', 'KNN or k-Nearest Neighbors',
                    'Gaussian Naive Bayes', 'Perceptron', 'Linear SVC', 
                    'Stochastic Gradient Descent', 'Decision Tree', 'Random Forest', 'Ensemble']

ensemble_models = [
    ('lrcv', LogisticRegressionCV(max_iter = 5000)),
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier(n_estimators=20)),
    ('knn', KNeighborsClassifier(n_neighbors = 4)),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('gnb',GaussianNB()),
]

index = ['Basic','Time_outlier','Time_/365','Clinic_outlier','Clinic_0-4(2steps)','Clinic_0-1(resizing)']

result_train_df = pd.DataFrame(columns=test_method_list)
result_test_df = pd.DataFrame(columns=test_method_list)
result_acc_test_df = pd.DataFrame(columns=test_method_list)
result_pre_test_df = pd.DataFrame(columns=test_method_list)
result_rec_test_df = pd.DataFrame(columns=test_method_list)
result_f1score_test_df = pd.DataFrame(columns=test_method_list)

In [17]:
'''
Model, predict and solve


- Methods
Logistic Regression
KNN or k-Nearest Neighbors
Support Vector Machines
Naive Bayes classifier
Decision Tree
Random Forrest
Perceptron
Artificial neural network
RVM or Relevance Vector Machine
Ensemble model
'''


def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average='weighted')
    recall = recall_score(y_test, pred, average='weighted')
    #     print('confusion matrix')
    #     print(confusion)
    print('accuracy:{}\nprecision:{}\nrecall:{}'.format(accuracy, precision, recall))
    
    return accuracy, precision, recall


def training():
    result_train = []
    result_test = []
    result_acc = []
    result_pre = []
    result_rec = []
    result_f1 = []
    for input_method in test_method_list:
        print('*** Method: ', input_method)

        if input_method == 'LogisticRegression':
            model = LogisticRegression(max_iter = 10000)

        elif input_method == 'Support Vector Machines':
            model = SVC()

        elif input_method == 'KNN or k-Nearest Neighbors':
            model = KNeighborsClassifier(n_neighbors = 4)

        elif input_method == 'Gaussian Naive Bayes':
            model = GaussianNB()

        elif input_method == 'Perceptron':
            model = Perceptron()

        elif input_method == 'Linear SVC':
            model = LinearSVC(max_iter=1000000)

        elif input_method == 'Stochastic Gradient Descent':
            model = SGDClassifier()

        elif input_method == 'Decision Tree':
            model = DecisionTreeClassifier()

        elif input_method == 'Random Forest':
            model = RandomForestClassifier(n_estimators=20)
            
        elif input_method == 'Ensemble':
            model  = VotingClassifier(ensemble_models, voting='soft') #, weights=[0]*len(ensemble_models)
    
        # Train
        model.fit(X_train, Y_train)
        acc_log = round(model.score(X_train, Y_train) * 100, 2)
        print('trained-acc: ', acc_log)
        result_train.append(acc_log)


        # Test
#         acc_log = round(model.score(X_test, Y_test) * 100, 2)
        print('test- ')
#         result_test.append(acc_log)
        
        # f1 score
        Y_pred = model.predict(X_test)
        acc, pre, rec = get_clf_eval(Y_test, Y_pred)
        result_acc.append(acc)
        result_pre.append(pre)
        result_rec.append(rec)
        f1 = f1_score(Y_test, Y_pred, average='weighted')
        print('f1 score:{}'.format(f1))
        result_f1.append(f1)
        
        print()
        print()

    result_train_df.loc[result_train_df.shape[0]] = result_train
#     result_test_df.loc[result_test_df.shape[0]] = result_test
    result_acc_test_df.loc[result_acc_test_df.shape[0]] = result_acc
    result_pre_test_df.loc[result_pre_test_df.shape[0]] = result_pre
    result_rec_test_df.loc[result_rec_test_df.shape[0]] = result_rec
    result_f1score_test_df.loc[result_f1score_test_df.shape[0]] = result_f1

In [18]:
'''
Basic Training & Test

'''

training()

*** Method:  LogisticRegression
trained-acc:  85.62
test- 
accuracy:0.435
precision:0.43892307692307697
recall:0.435
f1 score:0.4355051091846158


*** Method:  Support Vector Machines
trained-acc:  70.0
test- 
accuracy:0.485
precision:0.44589538739825146
recall:0.485
f1 score:0.46459803921568626


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  65.38
test- 
accuracy:0.495
precision:0.4900438596491228
recall:0.495
f1 score:0.4841293891812613


*** Method:  Gaussian Naive Bayes
trained-acc:  38.62
test- 
accuracy:0.285
precision:0.4842373579233588
recall:0.285
f1 score:0.3336797666152505


*** Method:  Perceptron
trained-acc:  64.88
test- 
accuracy:0.45
precision:0.42116005616005614
recall:0.45
f1 score:0.3982341601336589


*** Method:  Linear SVC


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  86.0
test- 
accuracy:0.425
precision:0.43592707454289736
recall:0.425
f1 score:0.42996592836749076


*** Method:  Stochastic Gradient Descent
trained-acc:  81.12
test- 
accuracy:0.44
precision:0.44318909538737133
recall:0.44
f1 score:0.43604805302402655


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.39
precision:0.4161903147042865
recall:0.39
f1 score:0.39955799195777836


*** Method:  Random Forest
trained-acc:  99.88
test- 
accuracy:0.42
precision:0.38863028841322483
recall:0.42
f1 score:0.4021508410200759


*** Method:  Ensemble


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.505
precision:0.4656332533013206
recall:0.505
f1 score:0.4838510378510378




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
'''
Add Time data

1) outlier value drop
2) clustering from step 0-N days to step 0-N/365 days
'''

'''
1)
'''
print('outlier of time: ')
print(df1.loc[df1['time'] < 0, 'time'], end='\n\n')

df1_outlier = df1.copy()
df1_outlier.loc[df1_outlier['time'] < 0, 'time'] = abs(df1_outlier.loc[df1_outlier['time'] < 0, 'time'])
print(df1_outlier.describe(), end='\n\n')

df = pd.concat([df5, df1_outlier,df2,df3,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']


'''
Add Time data(1) Training & Test

'''

training()

outlier of time: 
905   -7.945621
Name: time, dtype: float64

              time        event
count  1000.000000  1000.000000
mean     51.876125     0.891000
std      22.122689     0.311795
min       7.070708     0.000000
25%      37.401307     1.000000
50%      47.064712     1.000000
75%      60.966476     1.000000
max     217.078908     1.000000

['newlabel' 'time' 'Var1' 'Var2' 'Var3' 'Var4' 'Var5' 'Var6' 'Var7' 'Var8'
 'Var9' 'Var10' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6' 'G7' 'G8' 'G9' 'G10' 'G11'
 'G12' 'G13' 'G14' 'G15' 'G16' 'G17' 'G18' 'G19' 'G20' 'G21' 'G22' 'G23'
 'G24' 'G25' 'G26' 'G27' 'G28' 'G29' 'G30' 'G31' 'G32' 'G33' 'G34' 'G35'
 'G36' 'G37' 'G38' 'G39' 'G40' 'G41' 'G42' 'G43' 'G44' 'G45' 'G46' 'G47'
 'G48' 'G49' 'G50' 'G51' 'G52' 'G53' 'G54' 'G55' 'G56' 'G57' 'G58' 'G59'
 'G60' 'G61' 'G62' 'G63' 'G64' 'G65' 'G66' 'G67' 'G68' 'G69' 'G70' 'G71'
 'G72' 'G73' 'G74' 'G75' 'G76' 'G77' 'G78' 'G79' 'G80' 'G81' 'G82' 'G83'
 'G84' 'G85' 'G86' 'G87' 'G88' 'G89' 'G90' 'G91' 'G92' 'G93' 'G

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.59
precision:0.5469302124905621
recall:0.59
f1 score:0.5608035714285715


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  70.5
test- 
accuracy:0.515
precision:0.5130277185501066
recall:0.515
f1 score:0.49641016832703544


*** Method:  Gaussian Naive Bayes
trained-acc:  48.25
test- 
accuracy:0.33
precision:0.5143341773197448
recall:0.33
f1 score:0.3798331280303286


*** Method:  Perceptron
trained-acc:  69.12
test- 
accuracy:0.58
precision:0.5618969555035129
recall:0.58
f1 score:0.5638379431695896


*** Method:  Linear SVC
trained-acc:  94.62
test- 
accuracy:0.495
precision:0.49812933301458284
recall:0.495
f1 score:0.4964701332698715


*** Method:  Stochastic Gradient Descent
trained-acc:  77.75
test- 
accuracy:0.56
precision:0.5630086613949254
recall:0.56
f1 score:0.5606493837598778


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.42
precision:0.44583333333333336
recall:0.42
f1 score:0.4283130207685612


*** Method:  Random Forest
trained-acc:  99.88


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.56
precision:0.5195588235294117
recall:0.56
f1 score:0.5381746031746032




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
'''
2)
'''
df1_normalize = df1_outlier.copy()
df1_normalize['time'] = df1_normalize['time']/365.0
print(df1_normalize.describe(), end='\n\n')

df = pd.concat([df5, df1_normalize,df2,df3,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']


'''
Add Time data(1) Training & Test

'''

training()

              time        event
count  1000.000000  1000.000000
mean      0.142126     0.891000
std       0.060610     0.311795
min       0.019372     0.000000
25%       0.102469     1.000000
50%       0.128944     1.000000
75%       0.167031     1.000000
max       0.594737     1.000000

['newlabel' 'time' 'Var1' 'Var2' 'Var3' 'Var4' 'Var5' 'Var6' 'Var7' 'Var8'
 'Var9' 'Var10' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6' 'G7' 'G8' 'G9' 'G10' 'G11'
 'G12' 'G13' 'G14' 'G15' 'G16' 'G17' 'G18' 'G19' 'G20' 'G21' 'G22' 'G23'
 'G24' 'G25' 'G26' 'G27' 'G28' 'G29' 'G30' 'G31' 'G32' 'G33' 'G34' 'G35'
 'G36' 'G37' 'G38' 'G39' 'G40' 'G41' 'G42' 'G43' 'G44' 'G45' 'G46' 'G47'
 'G48' 'G49' 'G50' 'G51' 'G52' 'G53' 'G54' 'G55' 'G56' 'G57' 'G58' 'G59'
 'G60' 'G61' 'G62' 'G63' 'G64' 'G65' 'G66' 'G67' 'G68' 'G69' 'G70' 'G71'
 'G72' 'G73' 'G74' 'G75' 'G76' 'G77' 'G78' 'G79' 'G80' 'G81' 'G82' 'G83'
 'G84' 'G85' 'G86' 'G87' 'G88' 'G89' 'G90' 'G91' 'G92' 'G93' 'G94' 'G95'
 'G96' 'G97' 'G98' 'G99' 'G100' 'G101' 'G102' 'G103'

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.485
precision:0.44589538739825146
recall:0.485
f1 score:0.46459803921568626


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  65.75
test- 
accuracy:0.5
precision:0.4969140669966617
recall:0.5
f1 score:0.486035489924125


*** Method:  Gaussian Naive Bayes
trained-acc:  39.12
test- 
accuracy:0.315
precision:0.5325861197941335
recall:0.315
f1 score:0.37115562210389796


*** Method:  Perceptron
trained-acc:  77.12
test- 
accuracy:0.39
precision:0.4338712724057552
recall:0.39
f1 score:0.4107184265010352


*** Method:  Linear SVC
trained-acc:  89.62
test- 
accuracy:0.46
precision:0.4750069752694991
recall:0.46
f1 score:0.46736213235294133


*** Method:  Stochastic Gradient Descent
trained-acc:  78.12
test- 
accuracy:0.39
precision:0.4340350877192982
recall:0.39
f1 score:0.40531288951738115


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.415
precision:0.4273839009287926
recall:0.415
f1 score:0.4198351648351648


*** Method:  Random Forest
trained-acc:  100.

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.56
precision:0.5207430513288699
recall:0.56
f1 score:0.5377324788655576




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
'''
Correlating numerical features of Clinic data

- outlier value drop
- clustering from step 0-9 to step 0-4(clustering 2 steps)/0-1(just resizing)
'''


'''
1)
'''
df3_outlier = df3.copy()


# drop outlier
for col in df3_outlier.columns:
    for outlier in range(10,13):
        df3_outlier = df3_outlier.replace(outlier, 9)

# visualize
for col in df3_outlier.columns:
    print('#', col)
    print(df3_outlier[col].value_counts())
    print('-'*20)


df = pd.concat([df5, df1_outlier,df2,df3_outlier,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']



'''
Add Time data(1) +  Correlating Clinic data(1) 
Training & Test
'''

print()
training()

# Var1
2    235
3    204
1    171
4    139
5     95
0     57
6     50
7     27
8     13
9      9
Name: Var1, dtype: int64
--------------------
# Var2
3    221
2    218
4    163
1    113
5    109
6     65
0     48
7     30
9     20
8     13
Name: Var2, dtype: int64
--------------------
# Var3
2    260
3    196
1    156
4    130
5     97
0     55
6     55
7     23
8     16
9     12
Name: Var3, dtype: int64
--------------------
# Var4
2    242
3    195
1    150
4    140
5    106
6     67
0     36
7     32
8     16
9     16
Name: Var4, dtype: int64
--------------------
# Var5
2    247
3    223
4    161
5    124
1     76
6     63
7     41
0     28
9     19
8     18
Name: Var5, dtype: int64
--------------------
# Var6
2    240
3    212
4    128
1    127
5     99
6     64
0     53
7     40
8     20
9     17
Name: Var6, dtype: int64
--------------------
# Var7
1    269
2    208
3    144
0    128
4    118
5     62
6     47
7     16
8      6
9      2
Name: Var7, dtype: int64
--------------------

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  94.62
test- 
accuracy:0.5
precision:0.5063586413586414
recall:0.5
f1 score:0.5027788782259257


*** Method:  Stochastic Gradient Descent
trained-acc:  76.25
test- 
accuracy:0.57
precision:0.5448870043000478
recall:0.57
f1 score:0.5542871556856019


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.455
precision:0.46654750123701133
recall:0.455
f1 score:0.4591440992663708


*** Method:  Random Forest
trained-acc:  99.75
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.49
precision:0.45219548872180454
recall:0.49
f1 score:0.4690625


*** Method:  Ensemble
trained-acc:  100.0
test- 
accuracy:0.545
precision:0.5287424629535002
recall:0.545
f1 score:0.5284089912280702




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
'''
2-1) 2 steps clustering(0-4)
'''
df3_2c = df3_outlier.copy()

for col in df3_2c.columns:
    df3_2c.loc[df3_2c[col] <= 1, col] = 0
    df3_2c.loc[(df3_2c[col] > 1) & (df3_2c[col] <= 3), col] = 1
    df3_2c.loc[(df3_2c[col] > 3) & (df3_2c[col] <= 5), col] = 2
    df3_2c.loc[(df3_2c[col] > 5) & (df3_2c[col] <= 7), col] = 3
    df3_2c.loc[(df3_2c[col] > 7) & (df3_2c[col] <= 9), col] = 4

print(df3_2c.describe(), end='\n\n')

df = pd.concat([df5, df1_outlier,df2,df3_2c,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']


'''
Add Time data(1) +  Correlating Clinic data(2-1) 
Training & Test
'''

training()

              Var1         Var2         Var3         Var4         Var5  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean      1.226000     1.400000     1.256000     1.354000     1.500000   
std       0.958041     0.975167     0.968195     0.995829     0.942809   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       1.000000     1.000000     1.000000     1.000000     1.000000   
50%       1.000000     1.000000     1.000000     1.000000     1.000000   
75%       2.000000     2.000000     2.000000     2.000000     2.000000   
max       4.000000     4.000000     4.000000     4.000000     4.000000   

              Var6         Var7         Var8         Var9        Var10  
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  
mean      1.366000     0.933000     0.944000     0.930000     0.915000  
std       1.011468     0.947316     0.928292     0.917574     0.941627  
min       0.000000     0.000000     0.000

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  66.38
test- 
accuracy:0.52
precision:0.5057783564814815
recall:0.52
f1 score:0.49137629269565786


*** Method:  Linear SVC
trained-acc:  92.88
test- 
accuracy:0.465
precision:0.4777515381519176
recall:0.465
f1 score:0.470891071728569


*** Method:  Stochastic Gradient Descent
trained-acc:  75.88
test- 
accuracy:0.54
precision:0.5447348484848484
recall:0.54
f1 score:0.5127726219490234


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.475
precision:0.4772879365605659
recall:0.475
f1 score:0.4743722568180965


*** Method:  Random Forest
trained-acc:  100.0
test- 
accuracy:0.48
precision:0.44355906468266015
recall:0.48
f1 score:0.4579472140762463


*** Method:  Ensemble


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.555
precision:0.5178071253071254
recall:0.555
f1 score:0.5318910728910728




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
'''
2-2) just resizing(0-1)
'''
df3_rs = df3_outlier.copy()

for col in df3_rs.columns:
    df3_rs[col] = df3_rs[col]/9

print(df3_rs.describe(), end='\n\n')

df = pd.concat([df5, df1_outlier,df2,df3_rs,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

print(df.columns.values)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']




'''
Add Time data(1) +  Correlating Clinic data(2-2) 
Training & Test
'''

training()

              Var1         Var2         Var3         Var4         Var5  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean      0.328667     0.365889     0.332889     0.356333     0.387000   
std       0.207386     0.214559     0.210689     0.216080     0.210446   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.222222     0.222222     0.222222     0.222222     0.222222   
50%       0.333333     0.333333     0.333333     0.333333     0.333333   
75%       0.444444     0.444444     0.444444     0.444444     0.555556   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

              Var6         Var7         Var8         Var9        Var10  
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  
mean      0.358556     0.262111     0.265889     0.259667     0.262000  
std       0.223040     0.201703     0.202755     0.201437     0.201204  
min       0.000000     0.000000     0.000

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.585
precision:0.5442446524064171
recall:0.585
f1 score:0.5534275250035217


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  65.62
test- 
accuracy:0.5
precision:0.49104723502304154
recall:0.5
f1 score:0.4814138547549451


*** Method:  Gaussian Naive Bayes
trained-acc:  48.0
test- 
accuracy:0.33
precision:0.5143341773197448
recall:0.33
f1 score:0.3798331280303286


*** Method:  Perceptron
trained-acc:  8.88
test- 
accuracy:0.035
precision:0.3240947546531302
recall:0.035
f1 score:0.020877832512315272


*** Method:  Linear SVC
trained-acc:  94.12
test- 
accuracy:0.495
precision:0.5000427350427351
recall:0.495
f1 score:0.4969609824225876


*** Method:  Stochastic Gradient Descent
trained-acc:  50.38
test- 
accuracy:0.385
precision:0.5617104341736695
recall:0.385
f1 score:0.31890818257018355


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.41
precision:0.4327968460111318
recall:0.41
f1 score:0.4178859894377136


*** Method:  Random Forest
trained-acc:  99.7

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.56
precision:0.5427557603686636
recall:0.56
f1 score:0.5422987938596491




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print('*** train result ***', end='\n\n')
result_train_df.index = index
print(result_train_df)
print(end='\n\n\n')

# print('*** test result ***', end='\n\n')
# result_test_df.index = index
# print(result_test_df)
# print(end='\n\n\n')

print('*** test acc ***', end='\n\n')
result_acc_test_df.index = index
print(result_acc_test_df)
print(end='\n\n\n')

print('*** test pre ***', end='\n\n')
result_pre_test_df.index = index
print(result_pre_test_df)
print(end='\n\n\n')

print('*** test rec ***', end='\n\n')
result_rec_test_df.index = index
print(result_rec_test_df)
print(end='\n\n\n')

print('*** test f1 ***', end='\n\n')
result_f1score_test_df.index = index
print(result_f1score_test_df)
print(end='\n\n\n')

*** train result ***

                      LogisticRegression  Support Vector Machines  \
Basic                              85.62                    70.00   
Time_outlier                       95.25                    56.25   
Time_/365                          86.50                    70.00   
Clinic_outlier                     95.12                    56.25   
Clinic_0-4(2steps)                 93.50                    54.37   
Clinic_0-1(resizing)               92.88                    53.12   

                      KNN or k-Nearest Neighbors  Gaussian Naive Bayes  \
Basic                                      65.38                 38.62   
Time_outlier                               70.50                 48.25   
Time_/365                                  65.75                 39.12   
Clinic_outlier                             70.38                 48.00   
Clinic_0-4(2steps)                         68.25                 47.75   
Clinic_0-1(resizing)                       65.62  

In [25]:
print(result_train_df.describe().loc['max'])

LogisticRegression              95.25
Support Vector Machines         70.00
KNN or k-Nearest Neighbors      70.50
Gaussian Naive Bayes            48.25
Perceptron                      77.12
Linear SVC                      94.62
Stochastic Gradient Descent     81.12
Decision Tree                  100.00
Random Forest                  100.00
Ensemble                       100.00
Name: max, dtype: float64


In [26]:
# print(result_test_df.describe().loc['max'])

In [27]:
print(result_acc_test_df.describe().loc['max'])

LogisticRegression             0.545
Support Vector Machines        0.590
KNN or k-Nearest Neighbors     0.525
Gaussian Naive Bayes           0.330
Perceptron                     0.580
Linear SVC                     0.500
Stochastic Gradient Descent    0.570
Decision Tree                  0.475
Random Forest                  0.530
Ensemble                       0.560
Name: max, dtype: float64


In [28]:
print(result_pre_test_df.describe().loc['max'])

LogisticRegression             0.532783
Support Vector Machines        0.546930
KNN or k-Nearest Neighbors     0.517721
Gaussian Naive Bayes           0.532586
Perceptron                     0.561897
Linear SVC                     0.506359
Stochastic Gradient Descent    0.563009
Decision Tree                  0.477288
Random Forest                  0.491350
Ensemble                       0.542756
Name: max, dtype: float64


In [29]:
print(result_rec_test_df.describe().loc['max'])

LogisticRegression             0.545
Support Vector Machines        0.590
KNN or k-Nearest Neighbors     0.525
Gaussian Naive Bayes           0.330
Perceptron                     0.580
Linear SVC                     0.500
Stochastic Gradient Descent    0.570
Decision Tree                  0.475
Random Forest                  0.530
Ensemble                       0.560
Name: max, dtype: float64


In [30]:
print(result_f1score_test_df.describe().loc['max'])

LogisticRegression             0.538417
Support Vector Machines        0.560804
KNN or k-Nearest Neighbors     0.503383
Gaussian Naive Bayes           0.379833
Perceptron                     0.563838
Linear SVC                     0.502779
Stochastic Gradient Descent    0.560649
Decision Tree                  0.474372
Random Forest                  0.504603
Ensemble                       0.542299
Name: max, dtype: float64


In [31]:
'''
SMOTE at df1_outlier and df3_outlier

Add Time data(1) +  Correlating Clinic data(1) 
Training & Test

'''
from imblearn.over_sampling import SMOTE

df = pd.concat([df5, df1_outlier,df2,df3_outlier,df4], axis=1)
df = df.drop(['event', 'Treatment'], axis=1)

train_df, test_df = df.iloc[random_index[:800],:], df.iloc[random_index[800:1000],:]

X_train = train_df.drop(['newlabel'], axis=1)
Y_train = train_df['newlabel']

X_test = test_df.drop(['newlabel'], axis=1)
Y_test = test_df['newlabel']

smote = SMOTE(random_state=0)
X_train_over, Y_train_over = smote.fit_resample(X_train, Y_train)

# print()
# training()

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, Y_train.shape)
print('SMOTE 적용 전 레이블 값 분포: ')
print(pd.Series(Y_train).value_counts())
print('-'*100)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, Y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: ')
print(pd.Series(Y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (800, 311) (800,)
SMOTE 적용 전 레이블 값 분포: 
3    354
1    353
2     55
0     38
Name: newlabel, dtype: int64
----------------------------------------------------------------------------------------------------
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (1416, 311) (1416,)
SMOTE 적용 후 레이블 값 분포: 
0    354
1    354
2    354
3    354
Name: newlabel, dtype: int64


In [32]:
def training():
    result_train = []
    result_test = []
    result_acc = []
    result_pre = []
    result_rec = []
    result_f1 = []
    for input_method in test_method_list:
        print('*** Method: ', input_method)

        if input_method == 'LogisticRegression':
            model = LogisticRegression(max_iter = 10000)

        elif input_method == 'Support Vector Machines':
            model = SVC()

        elif input_method == 'KNN or k-Nearest Neighbors':
            model = KNeighborsClassifier(n_neighbors = 4)

        elif input_method == 'Gaussian Naive Bayes':
            model = GaussianNB()

        elif input_method == 'Perceptron':
            model = Perceptron()

        elif input_method == 'Linear SVC':
            model = LinearSVC(max_iter=1000000)

        elif input_method == 'Stochastic Gradient Descent':
            model = SGDClassifier()

        elif input_method == 'Decision Tree':
            model = DecisionTreeClassifier()

        elif input_method == 'Random Forest':
            model = RandomForestClassifier(n_estimators=20)
            
        elif input_method == 'Ensemble':
            model  = VotingClassifier(ensemble_models, voting='soft') #, weights=[0]*len(ensemble_models)
    
        # Train
        model.fit(X_train_over, Y_train_over)
        acc_log = round(model.score(X_train_over, Y_train_over) * 100, 2)
        print('trained-acc: ', acc_log)
        result_train.append(acc_log)


        # Test
        print('test- ')
        
        # f1 score
        Y_pred = model.predict(X_test)
        acc, pre, rec = get_clf_eval(Y_test, Y_pred)
        result_acc.append(acc)
        result_pre.append(pre)
        result_rec.append(rec)
        f1 = f1_score(Y_test, Y_pred, average='weighted')
        print('f1 score:{}'.format(f1))
        result_f1.append(f1)
        print()
        print()
        
        
    result_train_df.loc[result_train_df.shape[0]] = result_train
    #     result_test_df.loc[result_test_df.shape[0]] = result_test
    result_acc_test_df.loc[result_acc_test_df.shape[0]] = result_acc
    result_pre_test_df.loc[result_pre_test_df.shape[0]] = result_pre
    result_rec_test_df.loc[result_rec_test_df.shape[0]] = result_rec
    result_f1score_test_df.loc[result_f1score_test_df.shape[0]] = result_f1



In [33]:
'''
SMOTE

'''
training()

*** Method:  LogisticRegression
trained-acc:  96.54
test- 
accuracy:0.51
precision:0.5294708593114665
recall:0.51
f1 score:0.5181428571428572


*** Method:  Support Vector Machines
trained-acc:  54.45
test- 
accuracy:0.435
precision:0.5217662337662338
recall:0.435
f1 score:0.4681746544434583


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  54.94
test- 
accuracy:0.095
precision:0.5231521739130435
recall:0.095
f1 score:0.1120289303840348


*** Method:  Gaussian Naive Bayes
trained-acc:  72.46
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.52
precision:0.4799
recall:0.52
f1 score:0.49878009717961935


*** Method:  Perceptron
trained-acc:  64.55
test- 
accuracy:0.375
precision:0.4534753193331891
recall:0.375
f1 score:0.3054623425355133


*** Method:  Linear SVC
trained-acc:  95.62
test- 
accuracy:0.48
precision:0.5126807048400756
recall:0.48
f1 score:0.4944544241885282


*** Method:  Stochastic Gradient Descent
trained-acc:  55.23
test- 
accuracy:0.29
precision:0.5979305623872766
recall:0.29
f1 score:0.37500133863007107


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.375
precision:0.477758346651526
recall:0.375
f1 score:0.4128120687619483


*** Method:  Random Forest
trained-acc:  100.0
test- 
accuracy:0.485
precision:0.4911000303735952
recall:0.485
f1 score:0.4849010989010989


*** Method:  Ensemble
trained-acc:  100.0
test- 
accuracy:0.54
precision:0.5359611223799865
recall:0.54
f1 score:0.5347161051508877




In [34]:
'''
ADASYN
'''

from imblearn.over_sampling import ADASYN

ada = ADASYN(random_state=0, sampling_strategy  ='minority' )
X_train_over, Y_train_over = ada.fit_resample(X_train, Y_train)

# print()
# training()

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, Y_train.shape)
print('SMOTE 적용 전 레이블 값 분포: ')
print(pd.Series(Y_train).value_counts())
print('-'*100)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, Y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: ')
print(pd.Series(Y_train_over).value_counts())



print()
training()

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (800, 311) (800,)
SMOTE 적용 전 레이블 값 분포: 
3    354
1    353
2     55
0     38
Name: newlabel, dtype: int64
----------------------------------------------------------------------------------------------------
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (1110, 311) (1110,)
SMOTE 적용 후 레이블 값 분포: 
3    354
1    353
0    348
2     55
Name: newlabel, dtype: int64

*** Method:  LogisticRegression
trained-acc:  96.4
test- 
accuracy:0.52
precision:0.5210637689729942
recall:0.52
f1 score:0.519519459035588


*** Method:  Support Vector Machines
trained-acc:  55.95
test- 
accuracy:0.52
precision:0.5516637448453094
recall:0.52
f1 score:0.5316830530488091


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  45.68
test- 
accuracy:0.145
precision:0.38763553113553106
recall:0.145
f1 score:0.16618448847364511


*** Method:  Gaussian Naive Bayes
trained-acc:  73.51
test- 
accuracy:0.415
precision:0.4695440900562851
recall:0.415
f1 score:0.43617792642140474


*** Method:  Perceptron
trai

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Undefi

accuracy:0.465
precision:0.4755957622454595
recall:0.465
f1 score:0.4030267379679144


*** Method:  Linear SVC
trained-acc:  94.77
test- 
accuracy:0.46
precision:0.482745898257533
recall:0.46
f1 score:0.4700739589221783


*** Method:  Stochastic Gradient Descent
trained-acc:  80.54
test- 
accuracy:0.495
precision:0.5728351435745329
recall:0.495
f1 score:0.48409373615793766


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.415
precision:0.4438563793462869
recall:0.415
f1 score:0.42745890726946


*** Method:  Random Forest
trained-acc:  99.82
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.515
precision:0.5074672897196261
recall:0.515
f1 score:0.5020489849750008


*** Method:  Ensemble
trained-acc:  100.0
test- 
accuracy:0.555
precision:0.5353023504273504
recall:0.555
f1 score:0.5418661281742587




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
help(ADASYN)

Help on class ADASYN in module imblearn.over_sampling._adasyn:

class ADASYN(imblearn.over_sampling.base.BaseOverSampler)
 |  ADASYN(*, sampling_strategy='auto', random_state=None, n_neighbors=5, n_jobs=None)
 |  
 |  Oversample using Adaptive Synthetic (ADASYN) algorithm.
 |  
 |  This method is similar to SMOTE but it generates different number of
 |  samples depending on an estimate of the local distribution of the class
 |  to be oversampled.
 |  
 |  Read more in the :ref:`User Guide <smote_adasyn>`.
 |  
 |  Parameters
 |  ----------
 |  sampling_strategy : float, str, dict or callable, default='auto'
 |      Sampling information to resample the data set.
 |  
 |      - When ``float``, it corresponds to the desired ratio of the number of
 |        samples in the minority class over the number of samples in the
 |        majority class after resampling. Therefore, the ratio is expressed as
 |        :math:`\alpha_{os} = N_{rm} / N_{M}` where :math:`N_{rm}` is the
 |        number 

In [36]:
'''
SMOTETomek

'''
from imblearn.combine import SMOTETomek

smtom = SMOTETomek(random_state=139)
X_train_over, Y_train_over = smtom.fit_resample(X_train, Y_train)

# print()
# training()

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, Y_train.shape)
print('SMOTE 적용 전 레이블 값 분포: ')
print(pd.Series(Y_train).value_counts())
print('-'*100)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, Y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: ')
print(pd.Series(Y_train_over).value_counts())

training()

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (800, 311) (800,)
SMOTE 적용 전 레이블 값 분포: 
3    354
1    353
2     55
0     38
Name: newlabel, dtype: int64
----------------------------------------------------------------------------------------------------
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (1382, 311) (1382,)
SMOTE 적용 후 레이블 값 분포: 
2    354
0    353
1    338
3    337
Name: newlabel, dtype: int64
*** Method:  LogisticRegression
trained-acc:  97.32
test- 
accuracy:0.495
precision:0.5314094614963004
recall:0.495
f1 score:0.5111169765988202


*** Method:  Support Vector Machines
trained-acc:  53.91
test- 
accuracy:0.4
precision:0.5097702617198735
recall:0.4
f1 score:0.43969487499156024


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  54.92
test- 
accuracy:0.085
precision:0.5252218826924709
recall:0.085
f1 score:0.0929704790218915


*** Method:  Gaussian Naive Bayes
trained-acc:  73.01
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.5
precision:0.4615000000000001
recall:0.5
f1 score:0.4796275686093543


*** Method:  Perceptron
trained-acc:  73.59
test- 
accuracy:0.495
precision:0.5133188736681887
recall:0.495
f1 score:0.4563749017711419


*** Method:  Linear SVC
trained-acc:  96.53
test- 
accuracy:0.455
precision:0.4995618474662592
recall:0.455
f1 score:0.4721306330961504


*** Method:  Stochastic Gradient Descent
trained-acc:  61.22
test- 
accuracy:0.43
precision:0.5306625577812017
recall:0.43
f1 score:0.4551864144263746


*** Method:  Decision Tree
trained-acc:  100.0
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy:0.38
precision:0.4974558284852402
recall:0.38
f1 score:0.42247977285872024


*** Method:  Random Forest
trained-acc:  100.0
test- 
accuracy:0.475
precision:0.4698536656041727
recall:0.475
f1 score:0.46994534770429913


*** Method:  Ensemble
trained-acc:  100.0
test- 
accuracy:0.54
precision:0.5304709677419355
recall:0.54
f1 score:0.531688421561518




In [37]:
'''
SMOTEENN

'''

from imblearn.combine import SMOTEENN

smenn = SMOTEENN(random_state=139, sampling_strategy  ='all')
X_train_over, Y_train_over = smenn.fit_resample(X_train, Y_train)

# print()
# training()

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, Y_train.shape)
print('SMOTE 적용 전 레이블 값 분포: ')
print(pd.Series(Y_train).value_counts())
print('-'*100)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, Y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: ')
print(pd.Series(Y_train_over).value_counts())

training()

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (800, 311) (800,)
SMOTE 적용 전 레이블 값 분포: 
3    354
1    353
2     55
0     38
Name: newlabel, dtype: int64
----------------------------------------------------------------------------------------------------
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (705, 311) (705,)
SMOTE 적용 후 레이블 값 분포: 
2    349
0    342
3      9
1      5
Name: newlabel, dtype: int64
*** Method:  LogisticRegression
trained-acc:  98.72
test- 
accuracy:0.07
precision:0.6208869636963696
recall:0.07
f1 score:0.06386613585142997


*** Method:  Support Vector Machines
trained-acc:  77.87
test- 
accuracy:0.025
precision:0.002106125122830003
recall:0.025
f1 score:0.003802469135802469


*** Method:  KNN or k-Nearest Neighbors
trained-acc:  97.59
test- 


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Undefi

accuracy:0.04
precision:0.0032930756843800317
recall:0.04
f1 score:0.006074022829470771


*** Method:  Gaussian Naive Bayes
trained-acc:  54.04
test- 
accuracy:0.03
precision:0.000967741935483871
recall:0.03
f1 score:0.001875


*** Method:  Perceptron
trained-acc:  80.71
test- 
accuracy:0.025
precision:0.002019756200084069
recall:0.025
f1 score:0.003678977272727273


*** Method:  Linear SVC
trained-acc:  99.72
test- 
accuracy:0.085
precision:0.6766813067830278
recall:0.085
f1 score:0.08266247641247641


*** Method:  Stochastic Gradient Descent
trained-acc:  80.85
test- 
accuracy:0.04
precision:0.0029417879417879423
recall:0.04
f1 score:0.0054648625054561336


*** Method:  Decision Tree
trained-acc:  100.0
test- 
accuracy:0.085
precision:0.6158145917823932
recall:0.085
f1 score:0.11375236432540509


*** Method:  Random Forest
trained-acc:  100.0
test- 
accuracy:0.025
precision:0.002235576923076923
recall:0.025
f1 score:0.004097007223942209


*** Method:  Ensemble


C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


trained-acc:  100.0
test- 
accuracy:0.025
precision:0.0023200494284831637
recall:0.025
f1 score:0.004201416207710465




C:\Users\enssel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
help(SMOTEENN)

Help on class SMOTEENN in module imblearn.combine._smote_enn:

class SMOTEENN(imblearn.base.BaseSampler)
 |  SMOTEENN(*, sampling_strategy='auto', random_state=None, smote=None, enn=None, n_jobs=None)
 |  
 |  Over-sampling using SMOTE and cleaning using ENN.
 |  
 |  Combine over- and under-sampling using SMOTE and Edited Nearest Neighbours.
 |  
 |  Read more in the :ref:`User Guide <combine>`.
 |  
 |  Parameters
 |  ----------
 |  sampling_strategy : float, str, dict or callable, default='auto'
 |      Sampling information to resample the data set.
 |  
 |      - When ``float``, it corresponds to the desired ratio of the number of
 |        samples in the minority class over the number of samples in the
 |        majority class after resampling. Therefore, the ratio is expressed as
 |        :math:`\alpha_{os} = N_{rm} / N_{M}` where :math:`N_{rm}` is the
 |        number of samples in the minority class after resampling and
 |        :math:`N_{M}` is the number of samples in the m

In [39]:
index = ['Basic','Time_outlier','Time_/365','Clinic_outlier','Clinic_0-4(2steps)','Clinic_0-1(resizing)',
        'SMOTE', 'ADASYN', 'SMOTETomek', 'SMOTEENN']

print('*** train result ***', end='\n\n')
result_train_df.index = index
print(result_train_df)
print(end='\n\n\n')

# print('*** test result ***', end='\n\n')
# result_test_df.index = index
# print(result_test_df)
# print(end='\n\n\n')

print('*** test acc ***', end='\n\n')
result_acc_test_df.index = index
print(result_acc_test_df)
print(end='\n\n\n')

print('*** test pre ***', end='\n\n')
result_pre_test_df.index = index
print(result_pre_test_df)
print(end='\n\n\n')

print('*** test rec ***', end='\n\n')
result_rec_test_df.index = index
print(result_rec_test_df)
print(end='\n\n\n')

print('*** test f1 ***', end='\n\n')
result_f1score_test_df.index = index
print(result_f1score_test_df)
print(end='\n\n\n')

print('*** train acc describe ***', end='\n\n')
print(result_train_df.describe().loc['max'])

print('*** test acc describe ***', end='\n\n')
print(result_acc_test_df.describe().loc['max'])

print('*** test pre describe ***', end='\n\n')
print(result_pre_test_df.describe().loc['max'])

print('*** test rec describe ***', end='\n\n')
print(result_rec_test_df.describe().loc['max'])

print('*** test f1 describe ***', end='\n\n')
print(result_f1score_test_df.describe().loc['max'])

*** train result ***

                      LogisticRegression  Support Vector Machines  \
Basic                              85.62                    70.00   
Time_outlier                       95.25                    56.25   
Time_/365                          86.50                    70.00   
Clinic_outlier                     95.12                    56.25   
Clinic_0-4(2steps)                 93.50                    54.37   
Clinic_0-1(resizing)               92.88                    53.12   
SMOTE                              96.54                    54.45   
ADASYN                             96.40                    55.95   
SMOTETomek                         97.32                    53.91   
SMOTEENN                           98.72                    77.87   

                      KNN or k-Nearest Neighbors  Gaussian Naive Bayes  \
Basic                                      65.38                 38.62   
Time_outlier                               70.50                 48.25

LogisticRegression             0.620887
Support Vector Machines        0.551664
KNN or k-Nearest Neighbors     0.525222
Gaussian Naive Bayes           0.532586
Perceptron                     0.561897
Linear SVC                     0.676681
Stochastic Gradient Descent    0.597931
Decision Tree                  0.615815
Random Forest                  0.507467
Ensemble                       0.542756
Name: max, dtype: float64
*** test rec describe ***

LogisticRegression             0.545
Support Vector Machines        0.590
KNN or k-Nearest Neighbors     0.525
Gaussian Naive Bayes           0.520
Perceptron                     0.580
Linear SVC                     0.500
Stochastic Gradient Descent    0.570
Decision Tree                  0.475
Random Forest                  0.530
Ensemble                       0.560
Name: max, dtype: float64
*** test f1 describe ***

LogisticRegression             0.538417
Support Vector Machines        0.560804
KNN or k-Nearest Neighbors     0.503383
Gauss